In [0]:
from google.colab import drive
drive.mount('/gdrive')

%cd "/gdrive/My Drive/Day_Night"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/asd


In [0]:
from keras.applications import VGG16
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
from shutil import copyfile as cp

In [0]:
def readImg(path):
  img = cv2.imread(path)
  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LANCZOS4)
  img = img.reshape((1,224, 224, 3))
  return img

In [0]:
def test():
  if not os.path.exists("images/result"):
  os.makedirs("images/result/day")
  os.makedirs("images/result/night")
  img_list = os.listdir("images/test")
  for imgName in img_list:
    print(imgName)
    img = readImg("images/test/"+imgName)
    res = model.predict(img)
    if res[0][0] > res[0][1]:
      cp("images/test/"+imgName,"images/result/day/"+imgName)
    else:
      cp("images/test/"+imgName,"images/result/night/"+imgName)

In [0]:
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
pre_trained_model = vgg_conv
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False
for layer in pre_trained_model.layers[15:]:
    layer.trainable = True

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(512, activation="sigmoid")(x) 
x = Dropout(0.5)(x)
x = Dense(2, activation="softmax")(x)
opt = SGD(lr=0.00001)
model = Model(pre_trained_model.input, x)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    "images/train",
    target_size=(224, 224),
    batch_size = 5,
    shuffle=True,
    class_mode='categorical')

history = model.fit_generator(
    train_generator,
    epochs=20,
    steps_per_epoch=train_generator.n//5)